# Behavioral state from trial outcomes
This notebook describes how to load and process data from {cite}`Hulsey2024` to evaluate the behavioral state of mice performing perceptual decision tasks, given trial outcomes. It also shows how to evaluate the relation between these behavioral states and uninstructed movements (such as pupil, whisking, and locomotion).

## To do:
* Add citation to references.bib

### Environment Setup

In [29]:
try:
    from databook_utils.dandi_utils import dandi_download_open
except:
    !git clone https://github.com/AllenInstitute/openscope_databook.git
    %cd openscope_databook
    %pip install -e .

In [30]:
from dandi import dandiapi
from pynwb import NWBHDF5IO
import warnings

## Load NWB file with behavior data

In [31]:
dandiset_id = "000678"
dandi_filepath = "sub-BW046/sub-BW046_ses-20211110T084031.nwb"
download_loc = "."
dandi_api_key = None

In [32]:
# Ignore warnings from pynwb "UserWarning: Ignoring cached namespace..."
msg = "Ignoring cached namespace"
warnings.filterwarnings("ignore", message=msg)

In [33]:
# This can sometimes take a while depending on the size of the file
io = dandi_download_open(dandiset_id, dandi_filepath, download_loc, dandi_api_key=dandi_api_key)
nwb = io.read()

File already exists
Opening file


In [34]:
nwb

,start_time,stop_time,target_modality,cue_time,cue_ID,cue_duration,stimulus_type,stimulus_time,stimulus_duration,target_port,auditory_stim_id,auditory_stim_band,auditory_stim_difficulty,visual_stim_id,visual_stim_oreintation,visual_stim_difficulty,visual_gabor_angle,outcome,choice
id,,,,,,,,,,,,,,,,,,,
0,89.811790,91.119315,1.0,89.811790,0.0,0.0,0.0,89.811790,1200.0,1.0,2.0,1.0,NaN,1.0,0.0,0.0,90.0,1.0,1.0
1,99.180491,101.386792,1.0,99.180491,0.0,0.0,1.0,99.181489,1200.0,0.0,0.0,0.0,0.62,2.0,1.0,0.0,NaN,8.0,2.0
2,106.261617,108.470671,1.0,106.261617,0.0,0.0,1.0,106.262617,1200.0,1.0,1.0,1.0,0.98,2.0,1.0,0.0,NaN,8.0,2.0
3,114.374243,115.637905,1.0,114.374243,0.0,0.0,0.0,114.374243,1200.0,0.0,2.0,1.0,NaN,0.0,1.0,2.0,36.0,1.0,0.0
,start_time,stop_time,target_modality,cue_time,cue_ID,cue_duration,stimulus_type,stimulus_time,stimulus_duration,target_port,auditory_stim_id,auditory_stim_band,auditory_stim_difficulty,visual_stim_id,visual_stim_oreintation,visual_stim_difficulty,visual_gabor_angle,outcome,choice
id,,,,,,,,,,,,,,,,,,,
0,89.811790,91.119315,1.0,89.811790,0.0,0.0,0.0,89.811790,1200.0,1.0,2.0,1.0,NaN,1.0,0.0,0.0,90.0,1.0,1.0
1,99.180491,101.386792,1.0,99.180491,0.0,0.0,1.0,99.181489,1200.0,0.0,0.0,0.0,0.62,2.0,1.0,0.0,NaN,8.0,2.0
2,106.261617,108.470671,1.0,106.261617,0.0,0.0,1.0,106.262617,1200.0,1.0,1.0,1.0,0.98,2.0,1.0,0.0,NaN,8.0,2.0


In [35]:
trial_data = nwb.trials.to_dataframe()

In [38]:
trial_data

,start_time,stop_time,target_modality,cue_time,cue_ID,cue_duration,stimulus_type,stimulus_time,stimulus_duration,target_port,auditory_stim_id,auditory_stim_band,auditory_stim_difficulty,visual_stim_id,visual_stim_oreintation,visual_stim_difficulty,visual_gabor_angle,outcome,choice
id,,,,,,,,,,,,,,,,,,,
0,89.811790,91.119315,1.0,89.811790,0.0,0.0,0.0,89.811790,1200.0,1.0,2.0,1.0,NaN,1.0,0.0,0.0,90.0,1.0,1.0
1,99.180491,101.386792,1.0,99.180491,0.0,0.0,1.0,99.181489,1200.0,0.0,0.0,0.0,0.62,2.0,1.0,0.0,NaN,8.0,2.0
2,106.261617,108.470671,1.0,106.261617,0.0,0.0,1.0,106.262617,1200.0,1.0,1.0,1.0,0.98,2.0,1.0,0.0,NaN,8.0,2.0
3,114.374243,115.637905,1.0,114.374243,0.0,0.0,0.0,114.374243,1200.0,0.0,2.0,1.0,NaN,0.0,1.0,2.0,36.0,1.0,0.0
4,121.537456,123.743961,1.0,121.537456,0.0,0.0,1.0,121.537456,1200.0,1.0,1.0,1.0,0.75,2.0,1.0,0.0,NaN,8.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,6496.378067,6498.587819,1.0,6496.378067,0.0,0.0,1.0,6496.379067,1200.0,1.0,1.0,1.0,0.62,2.0,1.0,0.0,NaN,8.0,2.0
820,6504.699062,6506.950172,1.0,6504.699062,0.0,0.0,0.0,6504.700062,1200.0,1.0,2.0,1.0,NaN,1.0,0.0,1.0,72.0,16.0,2.0
821,6512.628310,6514.842901,1.0,6512.628310,0.0,0.0,1.0,6512.629307,1200.0,0.0,0.0,0.0,0.62,2.0,1.0,0.0,NaN,8.0,2.0
